In [3]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [4]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/LP_4_TheFarmersProblem/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 17:39:27.148373


## Step 1 - Generate Mathematical Formulation 

In [5]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [6]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [7]:
Markdown(response_1_text)

This problem can be formulated as a linear programming problem, a type of mathematical optimization model. Here's how to formulate it:

Parameters:
- Let Wp = $170 be the selling price of wheat per ton
- Let Cp = $150 be the selling price of corn per ton
- Let Bp1 = $36 be the selling price of sugar beets per ton for the first 6000 tons
- Let Bp2 = $10 be the selling price of sugar beets per ton for any amount over 6000 tons
- Let Wc = $150 be the cost to plant an acre of wheat
- Let Cc = $230 be the cost to plant an acre of corn
- Let Bc = $260 be the cost to plant an acre of sugar beets
- Let W = 2.5 be the yield of wheat per acre
- Let C = 3 be the yield of corn per acre
- Let B = 20 be the yield of sugar beets per acre
- Let L = 500 be the total acres of land available
- Let Wm = 200 tons be the minimum requirement for wheat
- Let Cm = 240 tons be the minimum requirement for corn
- Let Bq = 6000 tons be the sugar beet quota
- Let Ws = 1.4 be the selling to purchase price ratio for wheat
- Let Cs = 1.4 be the selling to purchase price ratio for corn

Decision Variables:
- Let w be the number of acres to devote to wheat
- Let c be the number of acres to devote to corn
- Let b be the number of acres to devote to sugar beets

Objective Function:
- Minimize the total costs (TC), which is the sum of the planting costs for each crop minus the total sales.

Constraints:
1. Total land constraint: w + c + b ≤ L
2. Wheat production constraint: W * w ≥ Wm
3. Corn production constraint: C * c ≥ Cm
4. Sugar beet production constraints:
   - B * b ≤ Bq (to ensure production does not exceed the quota)
   - B * (b - (Bq / B)) + Bq * (Bp1 - Bp2) ≤ Bp2 * B * (b - (Bq / B)) (to ensure sales are priced correctly based on the quota)
5. Non-negativity constraints: w, c, b ≥ 0

So, the mathematical optimization model would be:

Minimize: TC = Wc * w + Cc * c + Bc * b - Wp * W * w - Cp * C * c - (Bp1 * min(B * b, Bq) + Bp2 * max(0, B * b - Bq))

Subject to:
1. w + c + b ≤ L
2. W * w ≥ Wm
3. C * c ≥ Cm
4. B * b ≤ Bq
   B * (b - (Bq / B)) + Bq * (Bp1 - Bp2) ≤ Bp2 * B * (b - (Bq / B))
5. w, c, b ≥ 0

In [8]:
print(response_1_text)

This problem can be formulated as a linear programming problem, a type of mathematical optimization model. Here's how to formulate it:

Parameters:
- Let Wp = $170 be the selling price of wheat per ton
- Let Cp = $150 be the selling price of corn per ton
- Let Bp1 = $36 be the selling price of sugar beets per ton for the first 6000 tons
- Let Bp2 = $10 be the selling price of sugar beets per ton for any amount over 6000 tons
- Let Wc = $150 be the cost to plant an acre of wheat
- Let Cc = $230 be the cost to plant an acre of corn
- Let Bc = $260 be the cost to plant an acre of sugar beets
- Let W = 2.5 be the yield of wheat per acre
- Let C = 3 be the yield of corn per acre
- Let B = 20 be the yield of sugar beets per acre
- Let L = 500 be the total acres of land available
- Let Wm = 200 tons be the minimum requirement for wheat
- Let Cm = 240 tons be the minimum requirement for corn
- Let Bq = 6000 tons be the sugar beet quota
- Let Ws = 1.4 be the selling to purchase price ratio for 

## Step 2 - Generate the Pyomo Code

In [9]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [10]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [11]:
Markdown(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Sample data
Wp = 170
Cp = 150
Bp1 = 36
Bp2 = 10
Wc = 150
Cc = 230
Bc = 260
W = 2.5
C = 3
B = 20
L = 500
Wm = 200
Cm = 240
Bq = 6000
Ws = 1.4
Cs = 1.4

model = ConcreteModel()

# Decision variables
model.w = Var(within=NonNegativeReals)
model.c = Var(within=NonNegativeReals)
model.b = Var(within=NonNegativeReals)

# Objective function
def obj_func(model):
    return Wc * model.w + Cc * model.c + Bc * model.b - Wp * W * model.w - Cp * C * model.c - (Bp1 * min(B * model.b, Bq) + Bp2 * max(0, B * model.b - Bq))
model.obj = Objective(rule=obj_func, sense=minimize)

# Constraints
def land_constraint(model):
    return model.w + model.c + model.b <= L
model.con_land = Constraint(rule=land_constraint)

def wheat_constraint(model):
    return W * model.w >= Wm
model.con_wheat = Constraint(rule=wheat_constraint)

def corn_constraint(model):
    return C * model.c >= Cm
model.con_corn = Constraint(rule=corn_constraint)

def sugar_beet_qty_constraint(model):
    return B * model.b <= Bq
model.con_sugar_beet_qty = Constraint(rule=sugar_beet_qty_constraint)

def sugar_beet_price_constraint(model):
    return B * (model.b - (Bq / B)) + Bq * (Bp1 - Bp2) <= Bp2 * B * (model.b - (Bq / B))
model.con_sugar_beet_price = Constraint(rule=sugar_beet_price_constraint)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print('Number of acres to devote to wheat: ', model.w.value)
print('Number of acres to devote to corn: ', model.c.value)
print('Number of acres to devote to sugar beets: ', model.b.value)
print('Minimum total costs: ', model.obj())
```

Please note that you need to have Pyomo and a solver like GLPK installed to run this code. The code uses the GLPK solver, but you can replace it with other solvers like CPLEX, Gurobi, etc. based on your preference and availability.

In [12]:
print(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
from pyomo.environ import *

# Sample data
Wp = 170
Cp = 150
Bp1 = 36
Bp2 = 10
Wc = 150
Cc = 230
Bc = 260
W = 2.5
C = 3
B = 20
L = 500
Wm = 200
Cm = 240
Bq = 6000
Ws = 1.4
Cs = 1.4

model = ConcreteModel()

# Decision variables
model.w = Var(within=NonNegativeReals)
model.c = Var(within=NonNegativeReals)
model.b = Var(within=NonNegativeReals)

# Objective function
def obj_func(model):
    return Wc * model.w + Cc * model.c + Bc * model.b - Wp * W * model.w - Cp * C * model.c - (Bp1 * min(B * model.b, Bq) + Bp2 * max(0, B * model.b - Bq))
model.obj = Objective(rule=obj_func, sense=minimize)

# Constraints
def land_constraint(model):
    return model.w + model.c + model.b <= L
model.con_land = Constraint(rule=land_constraint)

def wheat_constraint(model):
    return W * model.w >= Wm
model.con_wheat = Constraint(rule=wheat_constraint)

def corn_constraint(model):
    return C * model.c >= Cm
model.con_corn = Constraint(

### Code Executability

In [13]:
from pyomo.environ import *

# Sample data
Wp = 170
Cp = 150
Bp1 = 36
Bp2 = 10
Wc = 150
Cc = 230
Bc = 260
W = 2.5
C = 3
B = 20
L = 500
Wm = 200
Cm = 240
Bq = 6000
Ws = 1.4
Cs = 1.4

model = ConcreteModel()

# Decision variables
model.w = Var(within=NonNegativeReals)
model.c = Var(within=NonNegativeReals)
model.b = Var(within=NonNegativeReals)

# Objective function
def obj_func(model):
    return Wc * model.w + Cc * model.c + Bc * model.b - Wp * W * model.w - Cp * C * model.c - (Bp1 * min(B * model.b, Bq) + Bp2 * max(0, B * model.b - Bq))
model.obj = Objective(rule=obj_func, sense=minimize)

# Constraints
def land_constraint(model):
    return model.w + model.c + model.b <= L
model.con_land = Constraint(rule=land_constraint)

def wheat_constraint(model):
    return W * model.w >= Wm
model.con_wheat = Constraint(rule=wheat_constraint)

def corn_constraint(model):
    return C * model.c >= Cm
model.con_corn = Constraint(rule=corn_constraint)

def sugar_beet_qty_constraint(model):
    return B * model.b <= Bq
model.con_sugar_beet_qty = Constraint(rule=sugar_beet_qty_constraint)

def sugar_beet_price_constraint(model):
    return B * (model.b - (Bq / B)) + Bq * (Bp1 - Bp2) <= Bp2 * B * (model.b - (Bq / B))
model.con_sugar_beet_price = Constraint(rule=sugar_beet_price_constraint)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print('Number of acres to devote to wheat: ', model.w.value)
print('Number of acres to devote to corn: ', model.c.value)
print('Number of acres to devote to sugar beets: ', model.b.value)
print('Minimum total costs: ', model.obj())

2024-05-22 17:47:22,585 ERROR pyomo.core: Rule failed when generating expression for Objective obj with index None:
PyomoException: Cannot convert non-constant Pyomo expression (6000  <  20*b) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.
2024-05-22 17:47:22,586 ERROR pyomo.core: Constructing component 'obj' from data=None failed:
PyomoException: Cannot convert non-constant Pyomo expression (6000  <  20*b) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y

PyomoException: Cannot convert non-constant Pyomo expression (6000  <  20*b) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### Solution Correctness